<center>

# **DSE 302: Data Science and Machine Learning**
# **Project 18**
## **Identification of Exoplanet Orbitals**

##### By

#### <font size="3">**Arihant Tiwari (19050) & Gourav Kumawat (19127)**</font>   


##### Instructor

#### <font size="3"> **Dr. Tanmay Basu** </font>
#####  <font size="2"> Assistant Professor, IISER Bhopal </font>

<br>

<img src="https://dse.iiserb.ac.in/img/IISERB_Logo.png" alt="drawing" width="100"/>
</center>

# **Importing the Data**

We will import the neccessary libraries, such as numpy for array calculations and functions, pandas for dataframe functoins, matplotlib for graphical plotting and imaging, and statistics for the median and mode calculation.

In [1]:
import numpy as np  # for array operations
import pandas as pd  # for dataframe operations
import matplotlib.pyplot as plt  # for plotting operations
import statistics as s  # for statitics functions
import sklearn

We will create three dataframes here:

- `train_data`: Stores the data values for the exoplanet candidates
- `train_labels`: Stores the labels for the exoplanet's semi-major axis
- `test_data`: Storese the test database

In [2]:
train_data = pd.read_csv("exoplanet_trn_data.csv")
train_labels = pd.read_csv("exoplanet_trn_data_targets.csv",
                           header=None)
test_data = pd.read_csv("exoplanet_tst_data.csv")

C:\Users\Gourav\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (20,33,34,81,82,106,107,153,154,155,170,172,204,280,281) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# **Data Processing**

The dataset contains data for various exoplanets, detected by the Kepler Exoplanet mission. The planets are "confirmed" exoplanet candidates

In [3]:
print("\nShape of the Training Data: ", train_data.shape)
print("Shape of the Test Data: ", test_data.shape, "\n")


Shape of the Training Data:  (17969, 288)
Shape of the Test Data:  (1997, 288) 



As we can observe, the data set contains a total of **17969** rows, which basically represent different exoplanets. Hence we have a total of 17969 candidates. Each of the exoplanet has a total of **288** features or parameteres which were recorded by the mission. These parameters are listed as columns. The test dataset contains **1997** number of samples and total of **288** features.

## Looking at our data

In [4]:
train_data["semi_major_axis"] = train_labels[1].values
train_data

,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,default_flag,sy_snum,sy_pnum,...,st_nspec,pl_nespec,pl_ntranspec,Unnamed: 282,Unnamed: 283,Unnamed: 284,Unnamed: 285,Unnamed: 286,Unnamed: 287,semi_major_axis
0,AU Mic c,AU Mic,c,HD 197481,HIP 102409,TIC 441420236,Gaia DR2 6794047652729201024,1,1,2,...,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.1101
1,Kepler-1476 b,Kepler-1476,b,NaN,NaN,TIC 164727404,Gaia DR2 2106597487787652352,0,1,1,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0932
2,Kepler-255 d,Kepler-255,d,NaN,NaN,TIC 272083669,Gaia DR2 2080202611655694336,0,1,3,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0202
3,Kepler-82 e,Kepler-82,e,NaN,NaN,TIC 63372898,Gaia DR2 2125850623586710400,0,1,5,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0632
4,K2-156 b,K2-156,b,NaN,NaN,TIC 176868262,Gaia DR2 3596250888028092160,1,1,1,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17964,Kepler-245 d,Kepler-245,d,NaN,NaN,TIC 63004906,Gaia DR2 2101768089124657664,0,1,4,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.1995
17965,Kepler-28 b,Kepler-28,b,NaN,NaN,TIC 63122956,Gaia DR2 2125731464014361088,1,1,2,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0620
17966,Kepler-1272 b,Kepler-1272,b,NaN,NaN,TIC 399794186,Gaia DR2 2100148611573075200,0,1,1,...,0,0,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0000
17967,HD 102117 b,HD 102117,b,HD 102117,HIP 57291,TIC 267519701,Gaia DR2 5341673851394297600,0,1,1,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.1532


## Checking Missing Data

As we can see in the above dataset, a lot of columns present have a `NaN` values which poses issues to our machine learning algorithms. We decided to remove the columns that have more than half i.e. $50\%$ of their rows as missing. But according to this article from [PressBooks](https://pressbooks.library.upei.ca/montelpare/chapter/working-with-missing-data/) where it is advised to remove all the columns that have more than $05\%$ of their values missing, which is ten times less than our initial estimate, we decided to drop those columns which have more than $95\%$ and $90\%$ of the total rows as `NaN` or missing.

In [5]:
missing_values_count = train_data.isnull().sum()
missing_values_count = missing_values_count.sort_values(ascending=False)
columns_with_missingvalues_morethan_05per = missing_values_count.head(
    207).index.values

In [6]:
tr05 = train_data.drop(columns=columns_with_missingvalues_morethan_05per, axis = 1).copy()

In the above cell, we created a code to form list, `columns_with_missingvalues_morethan_05per`, which contains a list of columns that have more than $95\%$ of the total values missing.

In [7]:
print("\nShape of the Training Data: ", train_data.shape)
print("Shape of the Tr05 Dataset ", tr05.shape,"\n")
print("Shape of the Test Dataset ", test_data.shape, "\n")


Shape of the Training Data:  (17969, 289)
Shape of the Tr05 Dataset  (17969, 82) 

Shape of the Test Dataset  (1997, 288) 



## Finding wrongly classified columns 

There are certain columns in the below given array which should belong to the numerical stature but are assigned an object denomination, due to the presence of impure values in them. We locate and mark such column names, *manually* and relist them as numerical values.

In [8]:
category_columns = tr05.select_dtypes(include=['object']).columns.tolist()
print("Total number of Object Type Columns: ",
      tr05[category_columns].nunique().sort_values(ascending=False).size)

Total number of Object Type Columns:  37


In [9]:
print("\nThe total number of Unique Values in the Respective Columns is\n")
print(tr05[category_columns].nunique().sort_values(ascending=False))


The total number of Unique Values in the Respective Columns is

pl_orbper          10769
pl_orbpererr2       7912
pl_orbpererr1       7097
sy_pm               6015
elat                4908
glat                4898
glon                4897
dec                 4895
decstr              4820
ra                  4720
pl_name             4484
hostname            3379
rastr               3306
tic_id              3256
gaia_id             3229
pl_orbperlim        1909
st_tefflim           884
releasedate          323
pl_pubdate           292
disc_pubdate         283
pl_nnotes            122
st_nphot             104
disc_instrument       95
disc_telescope        85
pl_ntranspec          73
disc_facility         65
st_nrvc               55
disc_locale           46
pl_nespec             39
st_nspec              24
rv_flag               19
ttv_flag              11
pl_controv_flag       11
discoverymethod       11
pul_flag               8
soltype                7
pl_letter              7
dtype: int

Now, we will use out domain expertise or field knowledge to find such columns that are definitely belonging to the numerical status.

In [10]:
wrong_classified_columns = [
    "pl_orbper", "pl_orbpererr1", "pl_orbpererr2", "sy_pm", "elat", "glat",
    "glon", "pl_orbperlim", "st_tefflim", "pl_ntranspec", "st_nspec"
]

Removing the above identified columns from the `Category Columns` so that we don't wrongly classify the needed columns as numeric into 'Object Domain'.

In [11]:
category_columns = list(set(category_columns) - set(wrong_classified_columns))
print("\nThe final list of Categorical Columns: \n")
category_columns


The final list of Categorical Columns: 



['disc_locale',
 'pl_nnotes',
 'pl_letter',
 'ttv_flag',
 'st_nrvc',
 'releasedate',
 'pl_nespec',
 'pl_pubdate',
 'tic_id',
 'dec',
 'decstr',
 'rv_flag',
 'pul_flag',
 'disc_telescope',
 'rastr',
 'discoverymethod',
 'ra',
 'hostname',
 'disc_facility',
 'st_nphot',
 'gaia_id',
 'pl_controv_flag',
 'disc_instrument',
 'soltype',
 'disc_pubdate',
 'pl_name']

## Converting Categorical Data to Numbers for Regression

Now we will convert the 'Categorical Data' to Numerical values to run our regression models.

In [12]:
for i in category_columns:
    tr05[i] = pd.Categorical(tr05[i])
    tr05[i] = tr05[i].cat.codes

**The Category Columns in the test and the train data are to be the same which can be seen below**

In [13]:
tr05[category_columns]

,disc_locale,pl_nnotes,pl_letter,ttv_flag,st_nrvc,releasedate,pl_nespec,pl_pubdate,tic_id,dec,...,ra,hostname,disc_facility,st_nphot,gaia_id,pl_controv_flag,disc_instrument,soltype,disc_pubdate,pl_name
0,40,7,1,4,0,303,13,278,2869,1826,...,4299,32,59,1,3169,0,81,6,240,43
1,40,1,0,4,0,71,13,176,1049,3691,...,2464,1610,29,0,1341,0,56,3,180,2039
2,40,1,2,4,0,37,13,164,1890,4023,...,3907,2212,29,0,713,0,56,2,180,2850
3,40,1,3,5,0,35,13,163,3130,3519,...,3494,2820,29,0,1464,0,56,1,153,3832
4,40,0,0,4,0,196,13,226,1200,2070,...,396,851,23,0,2438,0,56,6,202,1091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17964,40,1,2,5,0,37,13,164,3063,3426,...,3314,2201,29,0,1045,0,56,2,153,2819
17965,40,1,0,5,0,17,13,124,3088,3424,...,3386,2239,29,0,1454,0,56,6,131,2917
17966,40,77,0,6,0,220,13,241,2677,87,...,682,1385,29,1,920,0,56,6,180,1769
17967,37,7,0,4,48,17,13,59,1602,2056,...,349,317,31,1,2838,0,46,6,55,393


## Filling the Missing Data

As we can observe that there are a lot of `NaN` values in the columns, we will replace these values with the median of the data as it tends to be a more appropriate choice here, because the range of data is huge, and the extremeties are separated by a large gap.

In [14]:
def MedianCalculator(colname):
    pl_orbper = np.array(tr05[colname])
    pl_orbper = pl_orbper[np.logical_not(pd.isnull(pl_orbper))]
    m = []
    alpha = []
    for i in range(len(pl_orbper)):
        try:
            pl_orbper[i] = float(pl_orbper[i])
            m.append(pl_orbper[i])
        except:
            alpha.append(i)
    med = np.median(m)
    return med

for i in tr05.columns:
    if(i not in category_columns):
        tr05[i] = tr05[i].fillna(MedianCalculator(i))
    else:
        tr05[i] = tr05[i].fillna(s.mode(i))

In [15]:
#Checking if all the columns are free from the NaN values.
print("\nThe total number of NaN values in each column is: \n")
print(tr05.isnull().sum())


The total number of NaN values in each column is: 

pl_name            0
hostname           0
pl_letter          0
tic_id             0
gaia_id            0
                  ..
st_nrvc            0
st_nspec           0
pl_nespec          0
pl_ntranspec       0
semi_major_axis    0
Length: 82, dtype: int64


## Removing the non-numeric values in Columns with expected numeric values

As we can observe in the dataset, we can observe that there are certain non-numerical values in the dataset columns presetn, these are useless and will tend to produce errors while running our models. Hence, we will remove such columns.

In [16]:
def StringRowRemover(colname):
    arr = np.array(tr05[colname])
    alpha = []
    for i in range(len(arr)):
        try:
            arr[i] = float(arr[i])
        except:
            alpha.append(i)

    return alpha


for i in wrong_classified_columns:
    indexes = StringRowRemover(i)
    tr05 = tr05.drop(indexes)
    tr05 = tr05.set_index(np.arange(0, tr05.shape[0]))

In [17]:
tr05

,pl_name,hostname,pl_letter,tic_id,gaia_id,default_flag,sy_snum,sy_pnum,sy_mnum,cb_flag,...,sy_tmagerr2,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,semi_major_axis
0,43,32,1,2869,3169,1,1,2,0,0,...,-0.032000,278,303,7,1,0,1,13,0,0.1101
1,2039,1610,0,1049,1341,0,1,1,0,0,...,-0.006500,176,71,1,0,0,0,13,0,0.0932
2,2850,2212,2,1890,713,0,1,3,0,0,...,-0.007900,164,37,1,0,0,0,13,0,0.0202
3,3832,2820,3,3130,1464,0,1,5,0,0,...,-0.008800,163,35,1,0,0,0,13,0,0.0632
4,1091,851,0,1200,2438,1,1,1,0,0,...,-0.006100,226,196,0,0,0,0,13,0,0.0149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15771,1914,1503,1,896,1066,0,1,3,0,0,...,-0.007287,176,71,7,0,0,0,13,2,0.0905
15772,2819,2201,2,3063,1045,0,1,4,0,0,...,-0.007800,164,37,1,0,0,0,13,0,0.1995
15773,2917,2239,0,3088,1454,1,1,2,0,0,...,-0.006200,124,17,1,0,0,0,13,0,0.0620
15774,393,317,0,1602,2838,0,1,1,0,0,...,-0.006000,59,17,7,1,48,0,13,0,0.1532


### Finding the Correlation between Semi-Major Axis and the Variables

In [18]:
tr05.corr()["semi_major_axis"].sort_values()

discoverymethod   -0.175152
tran_flag         -0.074321
disc_locale       -0.053225
ra                -0.041538
sy_kmag           -0.040569
                     ...   
default_flag       0.052327
disc_telescope     0.061852
ima_flag           0.392637
semi_major_axis    1.000000
sy_mnum                 NaN
Name: semi_major_axis, Length: 71, dtype: float64

We are removing the column `sy_mnum` because it is giving `NaN` correlation with target variable `semi_major_axis`.

This shows that `sy_mnum` has only one constant value and hence irrelevant for our purpose.

In [19]:
tr05 = tr05.drop(columns = ["sy_mnum"], axis =1)

Checking the correlation again after removing the `sy_mnum` variable.

In [20]:
tr05.corr()["semi_major_axis"].sort_values()

discoverymethod   -0.175152
tran_flag         -0.074321
disc_locale       -0.053225
ra                -0.041538
sy_kmag           -0.040569
                     ...   
cb_flag            0.043226
default_flag       0.052327
disc_telescope     0.061852
ima_flag           0.392637
semi_major_axis    1.000000
Name: semi_major_axis, Length: 70, dtype: float64

## C-Map of the Dataset

In [21]:
tr05.corr().style.background_gradient(cmap='coolwarm')

In [22]:
tr05.corr().semi_major_axis

pl_name           -0.021931
hostname          -0.023302
pl_letter         -0.009631
tic_id             0.023881
gaia_id            0.032816
                     ...   
pl_nnotes          0.017089
st_nphot          -0.002634
st_nrvc           -0.003002
pl_nespec         -0.009203
semi_major_axis    1.000000
Name: semi_major_axis, Length: 70, dtype: float64

In [23]:
tr05.shape

(15776, 81)

As we can see above, for the target variable, i.e. The Semi-Major axis, the system calculates the Correlation with a total of 70 columns, whereas we can see that the total dataset contains about 81 columns including the Semi-Major axis. Hence there are some of the columns which are not being considered for Coorelation calculation.

It was discovered that the Correlation is NOT calculated for the Object Type and Categorical Columns in the Pandas dataframe. This arised an issue, as we converte some of the columns, few of which were crucial (by domain knowledge) were converted into Object types. Hence this discripancy occured in the number of columns and total number of Correlations calculated.

## Conversion of Object Data types into float

As discussed above, we will now convert the Object type data classes again into float.

In [24]:
for i in wrong_classified_columns:
    tr05[i] = tr05[i].astype("float")

In [25]:
tr05.corr().semi_major_axis.sort_values()

pl_orbpererr2     -0.792814
discoverymethod   -0.175152
tran_flag         -0.074321
elat              -0.058175
disc_locale       -0.053225
                     ...   
glon               0.068646
ima_flag           0.392637
pl_orbpererr1      0.792743
pl_orbper          0.792995
semi_major_axis    1.000000
Name: semi_major_axis, Length: 81, dtype: float64

This brings out a wonderful result, as we thought, the Orbital Period, or the `pl_orbper` should have a high correlation with the Dataset, and it actually shows up to have a value of 0.79 which is pretty reasonable.

## C-Map of the Dataset after Column Conversions

In [26]:
tr05.corr().style.background_gradient(cmap='coolwarm')

# Removing the Outliers

Several Columns in the dataset tend to contain contrasting data values, ranging from Mico range to Mega. Such difference in the dataset in way more than the acceptable range of the standard variance. Hence we removed the outliers in a hope to increase the correlation.

In [27]:
tr05[["pl_orbper", "semi_major_axis"]].describe()

,pl_orbper,semi_major_axis
count,1.577600e+04,15776.000000
mean,2.617040e+04,2.164164
std,3.201208e+06,75.368856
min,9.070629e-02,0.004400
25%,4.901447e+00,0.054200
50%,1.146347e+01,0.098800
75%,2.991414e+01,0.200100
max,4.020000e+08,7506.000000


In [28]:
non_category_columns = list(set(tr05.columns) - set(category_columns))
tr05_out = tr05.copy()

In [29]:
from scipy import stats

z_scores = stats.zscore(
    tr05_out[non_category_columns]
)  # Calculating the z_scores for each value in datafram
z_scores = np.abs(z_scores)  # taking only absolute value of z_score
filtered_entries = (z_scores < 10).all(
    axis=1)  # taking threshold to be |z_score| < 20
tr05_out = tr05_out[
    filtered_entries]  # considering only those entries where we dont have any outlier
tr05_out

,pl_name,hostname,pl_letter,tic_id,gaia_id,default_flag,sy_snum,sy_pnum,cb_flag,discoverymethod,...,sy_tmagerr2,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,semi_major_axis
0,43,32,1,2869,3169,1,1,2,0,9,...,-0.032000,278,303,7,1,0,1.0,13,0.0,0.1101
1,2039,1610,0,1049,1341,0,1,1,0,9,...,-0.006500,176,71,1,0,0,0.0,13,0.0,0.0932
2,2850,2212,2,1890,713,0,1,3,0,9,...,-0.007900,164,37,1,0,0,0.0,13,0.0,0.0202
3,3832,2820,3,3130,1464,0,1,5,0,9,...,-0.008800,163,35,1,0,0,0.0,13,0.0,0.0632
4,1091,851,0,1200,2438,1,1,1,0,9,...,-0.006100,226,196,0,0,0,0.0,13,0.0,0.0149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15771,1914,1503,1,896,1066,0,1,3,0,9,...,-0.007287,176,71,7,0,0,0.0,13,2.0,0.0905
15772,2819,2201,2,3063,1045,0,1,4,0,9,...,-0.007800,164,37,1,0,0,0.0,13,0.0,0.1995
15773,2917,2239,0,3088,1454,1,1,2,0,9,...,-0.006200,124,17,1,0,0,0.0,13,0.0,0.0620
15774,393,317,0,1602,2838,0,1,1,0,8,...,-0.006000,59,17,7,1,48,0.0,13,0.0,0.1532


Since we are not sure about the outcome of this operation, we form a new dataset called `t05_out`.

In [30]:
tr05_out[["pl_orbper", "semi_major_axis"]].describe()

,pl_orbper,semi_major_axis
count,15377.000000,15377.000000
mean,114.698792,0.294999
std,823.208543,0.826415
min,0.240104,0.005800
25%,4.897134,0.054200
50%,11.463474,0.097700
75%,29.221510,0.193100
max,43000.000000,25.000000


As we can see now, that the `std` is about six times the value of `mean` contrary to the previous, 150 times.

In [31]:
tr05_out

,pl_name,hostname,pl_letter,tic_id,gaia_id,default_flag,sy_snum,sy_pnum,cb_flag,discoverymethod,...,sy_tmagerr2,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,semi_major_axis
0,43,32,1,2869,3169,1,1,2,0,9,...,-0.032000,278,303,7,1,0,1.0,13,0.0,0.1101
1,2039,1610,0,1049,1341,0,1,1,0,9,...,-0.006500,176,71,1,0,0,0.0,13,0.0,0.0932
2,2850,2212,2,1890,713,0,1,3,0,9,...,-0.007900,164,37,1,0,0,0.0,13,0.0,0.0202
3,3832,2820,3,3130,1464,0,1,5,0,9,...,-0.008800,163,35,1,0,0,0.0,13,0.0,0.0632
4,1091,851,0,1200,2438,1,1,1,0,9,...,-0.006100,226,196,0,0,0,0.0,13,0.0,0.0149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15771,1914,1503,1,896,1066,0,1,3,0,9,...,-0.007287,176,71,7,0,0,0.0,13,2.0,0.0905
15772,2819,2201,2,3063,1045,0,1,4,0,9,...,-0.007800,164,37,1,0,0,0.0,13,0.0,0.1995
15773,2917,2239,0,3088,1454,1,1,2,0,9,...,-0.006200,124,17,1,0,0,0.0,13,0.0,0.0620
15774,393,317,0,1602,2838,0,1,1,0,8,...,-0.006000,59,17,7,1,48,0.0,13,0.0,0.1532


Some of the values in few columns were removed as a result of the above operation, hence leading to `NaN` as their Correlation with the `Semi-Major Axis`. To deal with this, we removed the columns that gave a `NaN` correlation with the Semi-Major axis after the Outlier removal.

In [32]:
tr05_out.corr().semi_major_axis.sort_values().tail(20)

sy_disterr1        0.214471
releasedate        0.218823
st_nspec           0.231739
glon               0.237599
soltype            0.238388
disc_facility      0.242979
rv_flag            0.279366
disc_telescope     0.296783
micro_flag         0.304490
pl_orbpererr1      0.506324
pl_orbper          0.830892
semi_major_axis    1.000000
cb_flag                 NaN
ptv_flag                NaN
ast_flag                NaN
etv_flag                NaN
ima_flag                NaN
dkin_flag               NaN
pl_orbperlim            NaN
st_tefflim              NaN
Name: semi_major_axis, dtype: float64

#### Removing the constant columns

This is the list of column that became `NaN` after removal of the outliers.

In [33]:
const_col = [
    'cb_flag', 'ptv_flag', 'ast_flag', 'etv_flag', 'ima_flag', 'dkin_flag',
    'pl_orbperlim', 'st_tefflim'
]
tr05_out = tr05_out.drop(columns=const_col, axis=1)

In [34]:
print("\nWITH OUTLIERS REMOVED\n\n",
      tr05_out.corr().semi_major_axis.sort_values().tail(20), "\n\n\n")
print("WITHOUT OUTLIERS REMOVED\n\n",
      tr05.corr().semi_major_axis.sort_values().tail(20))


WITH OUTLIERS REMOVED

 sy_vmagerr2        0.110916
pl_nnotes          0.118277
decstr             0.134863
sy_jmagerr1        0.173693
st_nrvc            0.174171
sy_hmagerr1        0.176934
default_flag       0.187995
sy_pm              0.199261
sy_disterr1        0.214471
releasedate        0.218823
st_nspec           0.231739
glon               0.237599
soltype            0.238388
disc_facility      0.242979
rv_flag            0.279366
disc_telescope     0.296783
micro_flag         0.304490
pl_orbpererr1      0.506324
pl_orbper          0.830892
semi_major_axis    1.000000
Name: semi_major_axis, dtype: float64 



WITHOUT OUTLIERS REMOVED

 dkin_flag          0.013505
pl_nnotes          0.017089
ast_flag           0.018554
disc_instrument    0.019104
pl_pubdate         0.022977
tic_id             0.023881
sy_gaiamagerr1     0.023932
decstr             0.026501
disc_facility      0.030787
gaia_id            0.032816
soltype            0.033198
releasedate        0.041698
cb_flag   

In [35]:
tr05_out.corr().style.background_gradient(cmap='coolwarm')

# Test Data Cleaning

In [36]:
total_columns = test_data.columns
columns_to_keep = tr05_out.columns
columns_to_remove = list(set(total_columns)-set(columns_to_keep))

for i in columns_to_remove:
        test_data = test_data.drop(columns=i,axis=1)

In [37]:
for i in category_columns:
    test_data[i] = pd.Categorical(test_data[i])
    test_data[i] = test_data[i].cat.codes

In [38]:
def MedianCalculator1(colname):
    pl_orbper = np.array(test_data[colname])
    pl_orbper = pl_orbper[np.logical_not(pd.isnull(pl_orbper))]
    m = []
    alpha = []
    for i in range(len(pl_orbper)):
        try:
            pl_orbper[i] = float(pl_orbper[i])
            m.append(pl_orbper[i])
        except:
            alpha.append(i)
    med = np.median(m)
    return med

        
for i in test_data.columns:
    if(i not in category_columns):
        test_data[i] = test_data[i].fillna(MedianCalculator1(i))
    else:
        test_data[i] = test_data[i].fillna(s.mode(i))

In [39]:
print("\nThe total number of NaN values in each column of Test data is: \n")
print(test_data.isnull().sum())


The total number of NaN values in each column of Test data is: 

pl_name         0
hostname        0
pl_letter       0
tic_id          0
gaia_id         0
               ..
st_nphot        0
st_nrvc         0
st_nspec        0
pl_nespec       0
pl_ntranspec    0
Length: 72, dtype: int64


In [40]:
def StringRowRemover1(colname):
    arr = np.array(test_data[colname])
    alpha = []
    for i in range(len(arr)):
        try:
            arr[i] = float(arr[i])
        except:
            alpha.append(i)

    return alpha


for i in test_data.columns:
    indexes = StringRowRemover1(i)
    test_data = test_data.drop(indexes)
    test_data = test_data.set_index(np.arange(0, test_data.shape[0]))

In [41]:
c = 'sy_mnum'
if c in test_data.columns:
    test_data = test_data.drop(columns = ["sy_mnum"], axis =1)

In [42]:
for i in test_data.columns:
    test_data[i] = test_data[i].astype("float")

# Feature Engineering

![SNOWFALL](https://th.bing.com/th/id/R.efe39a3a75e1ce0a199a32cdc1689ece?rik=2T2R6lsG6yJLVA&riu=http%3a%2f%2fwww.sliderbase.com%2fimages%2freferats%2f1148b%2f(25).PNG&ehk=o6h3a1sXrFXA7GrCaYHoGA9gPIksASYcz1dLa2350vw%3d&risl=&pid=ImgRaw&r=0)

In [69]:
tr05_out["pl_orbper1"] = tr05_out["pl_orbper"]**(2 / 3)
tr05_out["pl_orbper2"] = (tr05_out["pl_orbper"]**(2 / 3)) * (6.67 * 10**-11)
test_data["pl_orbper1"] = test_data["pl_orbper"]**(2 / 3)
test_data["pl_orbper2"] = (test_data["pl_orbper"]**(2 / 3)) * (6.67 * 10**-11)
tr05_out.insert(
    len(tr05_out.columns) - 1, 'semi_major_axis',
    tr05_out.pop('semi_major_axis'))

In [44]:
 tr05_out.corr().semi_major_axis.sort_values().tail(20), "\n\n\n"

(decstr             0.134863
 sy_jmagerr1        0.173693
 st_nrvc            0.174171
 sy_hmagerr1        0.176934
 default_flag       0.187995
 sy_pm              0.199261
 sy_disterr1        0.214471
 releasedate        0.218823
 st_nspec           0.231739
 glon               0.237599
 soltype            0.238388
 disc_facility      0.242979
 rv_flag            0.279366
 disc_telescope     0.296783
 micro_flag         0.304490
 pl_orbpererr1      0.506324
 pl_orbper          0.830892
 pl_orbper1         0.899004
 pl_orbper2         0.899004
 semi_major_axis    1.000000
 Name: semi_major_axis, dtype: float64,
 '\n\n\n')

In [70]:
tr05_out.to_csv('FinalData.csv', index = False)
test_data.to_csv('CleanedTestData.csv', index = False)

# Modelling 

In [46]:
from sklearn.model_selection import GridSearchCV  # Gridsearch for Parameter Tuning
from sklearn.pipeline import Pipeline  # Assembling steps to cross-validate together
from sklearn.metrics import mean_squared_error

In [47]:
tr_final = pd.read_csv('FinalData.csv')
tr_final.head()

,pl_name,hostname,pl_letter,tic_id,gaia_id,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,...,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,pl_orbper1,pl_orbper2,semi_major_axis
0,43,32,1,2869,3169,1,1,2,9,2021,...,303,7,1,0,1.0,13,0.0,7.085101,4.725763e-10,0.1101
1,2039,1610,0,1049,1341,0,1,1,9,2016,...,71,1,0,0,0.0,13,0.0,4.751911,3.169524e-10,0.0932
2,2850,2212,2,1890,713,0,1,3,9,2016,...,37,1,0,0,0.0,13,0.0,1.030188,6.871357e-11,0.0202
3,3832,2820,3,3130,1464,0,1,5,9,2014,...,35,1,0,0,0.0,13,0.0,3.265965,2.178399e-10,0.0632
4,1091,851,0,1200,2438,1,1,1,9,2018,...,196,0,0,0,0.0,13,0.0,0.871185,5.810802e-11,0.0149


In [48]:
final_columns = tr_final.columns
X = tr_final[[
    'pl_name', 'hostname', 'pl_letter', 'tic_id', 'gaia_id', 'default_flag',
    'sy_snum', 'sy_pnum', 'discoverymethod', 'disc_year', 'disc_pubdate',
    'disc_locale', 'disc_facility', 'disc_telescope', 'disc_instrument',
    'rv_flag', 'pul_flag', 'tran_flag', 'obm_flag', 'micro_flag', 'soltype',
    'pl_controv_flag', 'pl_orbper', 'pl_orbpererr1', 'pl_orbpererr2',
    'ttv_flag', 'rastr', 'ra', 'decstr', 'dec', 'glat', 'glon', 'elat', 'elon',
    'sy_pm', 'sy_pmerr1', 'sy_pmerr2', 'sy_pmra', 'sy_pmraerr1', 'sy_pmraerr2',
    'sy_pmdec', 'sy_pmdecerr1', 'sy_pmdecerr2', 'sy_dist', 'sy_disterr1',
    'sy_bmag', 'sy_bmagerr1', 'sy_bmagerr2', 'sy_vmag', 'sy_vmagerr1',
    'sy_vmagerr2', 'sy_jmag', 'sy_jmagerr1', 'sy_jmagerr2', 'sy_hmag',
    'sy_hmagerr1', 'sy_hmagerr2', 'sy_kmag', 'sy_kmagerr1', 'sy_kmagerr2',
    'sy_gaiamagerr1', 'sy_gaiamagerr2', 'sy_tmagerr1', 'sy_tmagerr2',
    'pl_pubdate', 'releasedate', 'pl_nnotes', 'st_nphot', 'st_nrvc',
    'st_nspec', 'pl_nespec', 'pl_ntranspec', 'pl_orbper1', 'pl_orbper2'
]].values
y = tr_final["semi_major_axis"].values

In [49]:
X.shape, y.shape

((15377, 74), (15377,))

### Linear Regression

In [50]:
from sklearn.linear_model import LinearRegression

In [51]:
linear = LinearRegression()
linear_parameters = {
    'fit_intercept': [True, False],    # Whether to calculate the intercept for this model 
    'normalize': [True, False],        # Whether the regressors X will be normalized before regression or not
    'copy_X': [True, False]            # If True, X will be copied; else, it may be overwritten.
}
grid_linear = GridSearchCV(linear, linear_parameters, scoring='neg_mean_squared_error', cv=5)
grid_linear.fit(X, y)
linear = grid_linear.best_estimator_
print("\nThe best parameters found for Linear Regression are: \n",
      grid_linear.best_params_)


The best parameters found for Linear Regression are: 
 {'copy_X': True, 'fit_intercept': True, 'normalize': False}


In [52]:
mean_squared_error(y,linear.predict(X))

0.06170949895096594

### Decision Tree Regression

In [53]:
from sklearn.tree import DecisionTreeRegressor

In [54]:
tree = DecisionTreeRegressor()
tree_parameters = {
    "max_depth": [1, 5, 9],                # maximum depth of the tree
    "min_samples_leaf": [1, 5, 10],        # minimum number of samples required to split an internal node
    "max_leaf_nodes": [10, 50, 100]        # maximum number of leaf nodes a decision tree can have
}
grid_tree = GridSearchCV(tree, tree_parameters, scoring='neg_mean_squared_error', cv=5)
grid_tree.fit(X, y)
tree = grid_tree.best_estimator_
print("\nThe best parameters found for Decision Tree Regression are: \n",
      grid_tree.best_params_)


The best parameters found for Decision Tree Regression are: 
 {'max_depth': 9, 'max_leaf_nodes': 100, 'min_samples_leaf': 1}


In [55]:
mean_squared_error(y, tree.predict(X))

0.013100348206523322

### Support Vector Regression

In [56]:
from sklearn.svm import SVR

In [57]:
svr = SVR()
svr_parameters = {
    'kernel' : ['rbf', 'sigmoid'],        # specifies the kernel type to be used in the algorithm 
    'C' : [1, 100],                       # regularization parameter
    'gamma' : [0.01, 0.1]                 # penalty parameter
}
grid_svr = GridSearchCV(svr, svr_parameters, scoring='neg_mean_squared_error', cv=5)
grid_svr.fit(X, y)
svr = grid_svr.best_estimator_
print("\nThe best parameters found for Support Vector Regression are: \n",
      grid_svr.best_params_)


The best parameters found for Support Vector Regression are: 
 {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}


In [58]:
mean_squared_error(y,svr.predict(X))

0.010523055080168767

### Random Forest Regression 

In [59]:
from sklearn.ensemble import RandomForestRegressor

In [60]:
forest = RandomForestRegressor()
forest_parameters = {
    "n_estimators": [10, 20],                  # number of trees in the forest
    "min_samples_split": [2, 10],               # minimum number of samples required to split an internal node
    "bootstrap": [True, False],                # whether bootstrap samples are used when building trees.
}
grid_forest = GridSearchCV(forest, forest_parameters, scoring='neg_mean_squared_error', cv=5)
grid_forest.fit(X, y)
forest = grid_forest.best_estimator_
print("\nThe best parameters found for Random Forest Regression are: \n",
      grid_forest.best_params_)


The best parameters found for Random Forest Regression are: 
 {'bootstrap': True, 'min_samples_split': 2, 'n_estimators': 20}


In [61]:
mean_squared_error(y,forest.predict(X))

0.007237825430819912

We can see that Random Forest Regression is giving least value of `mean_squared_error`. Hence, this is our best model and will be used to predict semi_major_axis for `test data`.

In [62]:
print(
    f"The r2 score for different regression models are as follows: \nLinear Regression: {linear.score(X,y)} \nDecision Tree Regression: {tree.score(X,y)} \nSupport Vector Regression: {svr.score(X,y)} \nRandom Forest Regression: {forest.score(X,y)}"
)

The r2 score for different regression models are as follows: 
Linear Regression: 0.9096385144532235 
Decision Tree Regression: 0.9808171036024436 
Support Vector Regression: 0.984591045046563 
Random Forest Regression: 0.9894016209955487


# Predicting semi-major axis from test data using best model

The best method from the above explored methods was the **Random Forest** model. Hence we use it to predict the final target variable.

In [71]:
X_test = pd.read_csv("CleanedTestData.csv").values

In [72]:
X_test.shape

(1773, 74)

In [73]:
tst_semi_major_axis = forest.predict(X_test)

In [74]:
np.savetxt('predicted_semi_major_axis.txt', tst_semi_major_axis, fmt='%f')    # Saving the prediction